In [1]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [2]:
import time
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.optimization import (
    run_deeplifting,
    run_differential_evolution,
    run_dual_annealing,
    run_ipopt,
    run_pygranso,
)

In [3]:
problem_names = [
    # 'ackley',
    # 'bukin_n6',
    # 'cross_in_tray',
    # 'drop_wave',
    # 'eggholder',
    # 'griewank',
    # 'holder_table',
    # 'levy',
    # 'levy_n13',
    # 'rastrigin',
    # 'schaffer_n2',
    # 'schaffer_n4',
    # 'schwefel',
    # 'shubert',
    # 'sine_envelope',
    'ackley_30d',
    'ex8_6_2',
]

In [ ]:
problem_performance_list = []
trials = 1

for problem_name in problem_names:
    print(problem_name)
    # Setup the problem
    problem = PROBLEMS_BY_NAME[problem_name]

    # Get the known minimum
    minimum_value = problem['global_minimum']

    # First run IPOPT
    outputs_ipopt = run_ipopt(problem, trials=trials)

    # # Get the final results for all IPOPT runs
    # ipopt_results = pd.DataFrame(
    #     outputs['final_results'],
    #     columns=['x1', 'x2', 'f', 'algorithm', 'time']
    # )
    # ipopt_results['problem_name'] = problem_name
    # ipopt_results['hits'] = np.where(
    #     np.abs(ipopt_results['f'] - minimum_value) <= 1e-4, 1, 0
    # )

    # # Add IPOPT to the problem_performance_list
    # problem_performance_list.append(ipopt_results)

    # Next add dual annealing
    outputs_dual_annealing = run_dual_annealing(problem, trials=trials)

    # # Get the final results for all dual annealing runs
    # dual_annleaing_results = pd.DataFrame(
    #     outputs['final_results'],
    #     columns=['x1', 'x2', 'f', 'algorithm', 'time']
    # )
    # dual_annleaing_results['problem_name'] = problem_name
    # dual_annleaing_results['hits'] = np.where(
    #     np.abs(dual_annleaing_results['f'] - minimum_value) <= 1e-4, 1, 0
    # )

    # # Add dual annealing to the problem_performance_list
    # problem_performance_list.append(dual_annleaing_results)

    # Next add differential evolution
    outputs_differential_evolution = run_differential_evolution(problem, trials=trials)

    # # Get the final results for all differential evolution runs
    # differential_evolution_results = pd.DataFrame(
    #     outputs['final_results'],
    #     columns=['x1', 'x2', 'f', 'algorithm', 'time']
    # )
    # differential_evolution_results['problem_name'] = problem_name
    # differential_evolution_results['hits'] = np.where(
    #     np.abs(differential_evolution_results['f'] - minimum_value) <= 1e-4, 1, 0
    # )

    # # Add differential evolution to the problem_performance_list
    # problem_performance_list.append(differential_evolution_results)

    # Next add pygranso
    outputs = run_pygranso(problem, trials=trials)

    # # Get the final results for all differential evolution runs
    # pygranso_results = pd.DataFrame(
    #     outputs['final_results'],
    #     columns=['x1', 'x2', 'f', 'algorithm', 'time']
    # )
    # pygranso_results['problem_name'] = problem_name
    # pygranso_results['hits'] = np.where(
    #     np.abs(pygranso_results['f'] - minimum_value) <= 1e-4, 1, 0
    # )

    # # Add differential evolution to the problem_performance_list
    # problem_performance_list.append(pygranso_results)

    # Finally, add deeplifting
    outputs = run_deeplifting(
        problem,
        hidden_sizes=(128, 128),
        output_activation='leaky_relu',
        problem_name=problem_name,
        trials=trials,
    )

    # # Get the final results for all differential evolution runs
    # deeplifting_results = pd.DataFrame(
    #     outputs['final_results'],
    #     columns=['x1', 'x2', 'f', 'algorithm', 'time']
    # )
    # deeplifting_results['problem_name'] = problem_name
    # deeplifting_results['hits'] = np.where(
    #     np.abs(deeplifting_results['f'] - minimum_value) <= 1e-4, 1, 0
    # )

    # # Add differential evolution to the problem_performance_list
    # problem_performance_list.append(deeplifting_results)

ackley_30d

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

n_gpus: 0


╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
════════════════════════════════════════════════════════════════════════════════════════════════════

/Users/ryandevera/.virtualenvs/deeplifing/lib/python3.9/site-packages/pygranso/private/bfgsHessianInverseLimitedMem.py:237: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3575.)
  alpha[j,:]  = self.rho[0,j] * (self.S[:,j].T  @ q)


   2 ║ 1.000000 │  19.9128299341 ║  19.9128299341 ║ 0.000000 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.375269   ║ 
   3 ║ 1.000000 │  19.6897586017 ║  19.6897586017 ║ 0.000000 │   -  ║ S  │     3 │ 0.250000 ║     1 │ 0.356338   ║ 
   4 ║ 1.000000 │  19.5931063690 ║  19.5931063690 ║ 0.000000 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.664405   ║ 
   5 ║ 1.000000 │  19.3727354104 ║  19.3727354104 ║ 0.000000 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.055459   ║ 
   6 ║ 1.000000 │  19.3663612775 ║  19.3663612775 ║ 0.000000 │   -  ║ S  │     2 │ 0.500000 ║     1 │ 0.042169   ║ 
   7 ║ 1.000000 │  19.3652179988 ║  19.3652179988 ║ 0.000000 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.027132   ║ 
   8 ║ 1.000000 │  19.3645676169 ║  19.3645676169 ║ 0.000000 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.016345   ║ 
   9 ║ 1.000000 │  19.3645002572 ║  19.3645002572 ║ 0.000000 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.007886   ║ 
  10 ║ 1.000000 │  19.3644150860 ║  19.3644150860 ║ 0.000000 │   -  ║ S 

In [ ]:
# results_df = pd.concat(problem_performance_list, ignore_index=True)

In [ ]:
# performance_df = (
#     results_df
#     .groupby(['problem_name', 'algorithm'])
#     .agg({'hits': 'mean', 'time': 'sum'})
#     .reset_index()
#     .rename(columns={'hits': 'Success Rate', 'time': 'Total Run Time'})
# )

In [ ]:
# performance_df

In [ ]:
outputs['results'][1][:20, :]

In [ ]:
# performance_df = (
#     performance_df
#     .pivot_table(
#         index='problem_name',
#         columns='algorithm',
#         values=['Success Rate', 'Total Run Time']
#     )
#     .T
#     .reorder_levels(['algorithm', None])
#     .sort_index()
#     .T
# )

In [ ]:
# performance_df = performance_df.apply(lambda x: np.round(x, 3))

In [ ]:
# performance_df.to_csv('/Users/ryandevera/Desktop/deeplifting_results.csv')

In [ ]:
# performance_df

In [ ]:
outputs_ipopt

In [ ]:
index = 1
ipopt_outputs = outputs_ipopt['results'][index, :, :]

In [ ]:
ipopt_outputs = ipopt_outputs[~np.isnan(ipopt_outputs).any(axis=1)]

In [ ]:
midpoint = len(ipopt_outputs) // 2
ipopt_outputs = ipopt_outputs[[0, midpoint, -1], :2]

In [ ]:
ipopt_outputs = ipopt_outputs.tolist()

In [ ]:
ipopt_outputs

In [ ]:
dual_annealing_outputs = outputs_dual_annealing['callbacks'][index].x_history
differential_evolution_outputs = outputs_differential_evolution['callbacks'][
    index
].x_history

In [ ]:
trajectories = [
    ipopt_outputs,
    dual_annealing_outputs,
    differential_evolution_outputs,
]

models = ['IPOPT', 'Dual Annealing', 'Differential Evolution']

In [ ]:
from deeplifting.utils import create_contour_plot

In [ ]:
fig = create_contour_plot(
    problem_name=problem_name,
    problem=problem,
    models=models,
    trajectories=trajectories,
    colormap='OrRd_r',
)

In [ ]:
from cyipopt import minimize_ipopt

In [ ]:
?minimize_ipopt

In [ ]:
import numpy as np

problem = PROBLEMS_BY_NAME['sine_envelope']

In [ ]:
results = np.zeros((1, 1, 3))
trial = 0
objective = problem['objective']

In [ ]:
objective(np.array([0.0, 0.0]), results=results, trial=trial)

In [ ]:
def sine_envelope_2d(x, y):
    """
    Compute the 2D sine envelope function value.

    :param x: x-coordinate or numpy array of x-coordinates.
    :param y: y-coordinate or numpy array of y-coordinates.
    :return: Value(s) of the sine envelope function.
    """
    r = np.sqrt(x**2 + y**2)
    numerator = np.sin(r) ** 2 - 0.5
    denominator = (1 + 0.001 * r**2) ** 2
    return 0.5 + numerator / denominator

In [ ]:
sine_envelope_2d(0, 0)